# Postprocessing MMC case

In [1]:
# Load the modules

# Add any possible locations of amr-wind-frontend here
amrwindfedirs = ['/projects/wind_uq/lcheung/amrwind-frontend/',
                 '/ccs/proj/cfd162/lcheung/amrwind-frontend/']
import sys, os, shutil
for x in amrwindfedirs: sys.path.insert(1, x)
import MMCutil as writeMMC
import postproamrwindsample_xarray as ppsamplexr

# Load the libraries
import numpy as np
import pandas as pd
import amrwind_frontend as amrwind
import matplotlib.pyplot    as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.interpolate import CubicSpline

# Also ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Make all plots inline 
%matplotlib inline

/ascldap/users/lcheung/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/ascldap/users/lcheung/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
def find_nearest(a, a0):
    "Element in nd array `a` closest to the scalar value `a0`"
    idx = np.abs(a - a0).argmin()
    return idx

In [3]:
rundir = '/nscratch/lcheung/2024/HPP_WindSolar/MMC_HPP10'

In [4]:
# Start the amrwind_frontend app 
case = amrwind.MyApp.init_nogui()

TclError: couldn't connect to display ":616822712"

In [ ]:
case.ABLpostpro_loadnetcdffile(rundir+'/post_processing/abl_statistics00000.nc')

In [ ]:
zMMC     = np.loadtxt('MMC_all_heights.dat')
tMMC     = np.loadtxt('MMC_time_sec.dat')
uMMC     = np.loadtxt('MMC_Ublend.dat')
vMMC     = np.loadtxt('MMC_Vblend.dat')

twindow = 600
avgtimes = [[1*3600-twindow, 1*3600],
            [2*3600-twindow, 2*3600],
            [4*3600-twindow, 4*3600],
            [6*3600-twindow, 6*3600],
            [8*3600-twindow, 8*3600],            
            [10*3600-twindow, 10*3600],            
           ]

In [ ]:
# Plot velocity
fig, axs = plt.subplots(1,len(avgtimes),figsize=(2.25*len(avgtimes),4), facecolor='w', dpi=150, sharey=True)
for it, avgt in enumerate(avgtimes):
    ax=axs[it]
    dat=case.ABLpostpro_plotprofiles(ax=ax, plotvars=['Uhoriz'], avgt=avgt, doplot=False)
    ax.plot(dat['Uhoriz']['data'], dat['Uhoriz']['z'], label='AMR-Wind')
    
    # Get the nearest MMC data point
    targett = avgt[1]
    iclosest = find_nearest(tMMC, targett)
    uMMC_t   = uMMC[iclosest, :]
    vMMC_t   = vMMC[iclosest, :]
    uh_MMC   = np.sqrt(uMMC_t**2 + vMMC_t**2)
    ax.plot(uh_MMC, zMMC, 'k--', label='Data')
    
    #ax.plot(np.sqrt(np.array(profile['u'])**2 + np.array(profile['v'])**2), zMMC, 'k--', label='NOW23')
    ax.set_xlabel('U [m/s]')
    ax.set_title('t=%.0fs'%avgt[1])
    ax.grid(linestyle=':')
    ax.set_xlim([1, 14])
    
axs[0].set_ylabel('z [m]')
axs[0].set_ylim([0, 960])
axs[0].legend(prop={'size':8})

fig.suptitle('Horizontal velocity [10-min avg]')

In [ ]:
# Plot TKE
fig, axs = plt.subplots(1,len(avgtimes),figsize=(2.25*len(avgtimes),4), facecolor='w', dpi=150, sharey=True)
for it, avgt in enumerate(avgtimes):
    ax=axs[it]
    dat=case.ABLpostpro_plotprofiles(ax=ax, plotvars=['TKE'], avgt=avgt, doplot=False)
    ax.plot(dat['TKE']['data'], dat['TKE']['z'], label='AMR-Wind')
    #ax.plot(np.sqrt(np.array(profile['u'])**2 + np.array(profile['v'])**2), zMMC, 'k--')
    ax.set_xlabel('TKE [$m^2/s^2$]')
    ax.set_title('t=%.0fs'%avgt[1])
    #ax.set_xlim([0, 0.3])
    ax.grid(linestyle=':')
    

axs[0].set_ylabel('z [m]')
axs[0].set_ylim([0, 960])
axs[0].legend(prop={'size':8})

fig.suptitle('TKE profiles [10-min avg]')

In [ ]:
# Plot TKE
fig, axs = plt.subplots(1,len(avgtimes),figsize=(2.25*len(avgtimes),4), facecolor='w', dpi=150, sharey=True)
for it, avgt in enumerate(avgtimes):
    ax=axs[it]
    dat=case.ABLpostpro_plotprofiles(ax=ax, plotvars=['TI_TKE'], avgt=avgt, doplot=False)
    ax.plot(dat['TI_TKE']['data'], dat['TI_TKE']['z'], label='AMR-Wind')
    #ax.plot(np.sqrt(np.array(profile['u'])**2 + np.array(profile['v'])**2), zMMC, 'k--')
    ax.set_xlabel('TI [-]')
    ax.set_title('t=%.0fs'%avgt[1])
    #ax.set_xlim([0, 0.3])
    ax.grid(linestyle=':')
    

axs[0].set_ylabel('z [m]')
axs[0].set_ylim([0, 960])
axs[0].legend(prop={'size':8})

fig.suptitle('TI profiles [10-min avg]')

In [ ]:
# Plot Temperature
fig, axs = plt.subplots(1,len(avgtimes),figsize=(2.25*len(avgtimes),4), facecolor='w', dpi=150, sharey=True)
for it, avgt in enumerate(avgtimes):
    ax=axs[it]
    dat=case.ABLpostpro_plotprofiles(ax=ax, plotvars=['Temperature'], avgt=avgt, doplot=False)
    ax.plot(dat['T']['data'], dat['T']['z'], label='AMR-Wind')
    #ax.plot(np.sqrt(np.array(profile['u'])**2 + np.array(profile['v'])**2), zMMC, 'k--')
    ax.set_xlabel('T [K]')
    ax.set_title('t=%.0fs'%avgt[1])
    ax.grid(linestyle=':')
    ax.set_xlim([278, 290])
    
axs[0].set_ylabel('z [m]')
axs[0].set_ylim([0, 960])
axs[0].legend(prop={'size':8})

fig.suptitle('Temperature profiles [10-min avg]')

In [ ]:
# Plot Wind direction
fig, axs = plt.subplots(1,len(avgtimes),figsize=(2.25*len(avgtimes),4), facecolor='w', dpi=150, sharey=True)
for it, avgt in enumerate(avgtimes):
    ax=axs[it]
    dat=case.ABLpostpro_plotprofiles(ax=ax, plotvars=['WindDir'], avgt=avgt, doplot=False)
    ax.plot(dat['WindDir']['data'], dat['WindDir']['z'], label='AMR-Wind')
    print(np.interp(90, dat['WindDir']['z'], dat['WindDir']['data']))
    #ax.plot(np.sqrt(np.array(profile['u'])**2 + np.array(profile['v'])**2), zMMC, 'k--')
    ax.set_xlabel('Wind Dir [deg]')
    ax.set_title('t=%.0fs'%avgt[1])
    ax.grid(linestyle=':')
    #ax.set_xlim([0, 0.3])
    
axs[0].set_ylabel('z [m]')
axs[0].set_ylim([0, 960])
axs[0].legend(prop={'size':8})

fig.suptitle('Wind Direction [10-min avg]')